Here we are going to extract the basketball seasons from the [NBA Site](https://www.nba.com/stats/teams/traditional?Outcome=&SeasonType=Regular+Season&Season=2023-24).

1. To extract dataset from the site, we are going to use a web scraper, and chosen to go with Selenium. <br/>
   Read the docs to know more about [Selenium](https://selenium-python.readthedocs.io/installation.html)


In [83]:
# Importing all neccesary packages
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

import pandas as pd
from openpyxl import Workbook, load_workbook
from buckets import dimes

In [84]:
# Webdriver: Chrome | Site: NBA site
driver = webdriver.Chrome()
url = "https://www.nba.com/stats/teams/traditional/"
driver.get(url)

In [85]:
# Create an Excel to begin data transfer
wb = Workbook()
wb.save('nba_stats.xlsx')
ws = wb.active
dfs = []

In [86]:
# FIX!! SUPPOSED TO EXTRACT THE DEFINITION OF EACH COLUMN

# abbr_list = {}

# glossary_btn = Select(driver.find_element(By.CLASS_NAME, "IconButton_iconLink__Erl8y Crom_cromGlossary__ZHrZf"))
# glossary_btn.click()

# for li in list:
# abbr_list[abbr] = 

In [87]:
# Capture all seasons in the list from web page
season_drpdwn = Select(driver.find_element(By.CLASS_NAME,"DropDown_select__4pIg9"))
seasons_arr = [sn.text for sn in season_drpdwn.options]
xpath_table = dimes.get('XPATH')

In [88]:
# Fetch and Export Seasons' dataset into Excel
# We created multiple df requirinto insert seperate sheets in Excel. 
# Primary ('Sheet') is the junction of all df and is focused for this project,
# so we convert into CSV for its performance and simplicity

for i,sn in enumerate(seasons_arr):
    season_drpdwn.select_by_visible_text(sn)
    table = WebDriverWait(driver, 40).until(EC.visibility_of_element_located((By.XPATH, xpath_table))).get_attribute("outerHTML")
    df = pd.read_html(table)[0]
    df.dropna(how='all', axis=1, inplace=True)
    df.columns.values[0] = 'Rank'
    df.insert(2, "Season", sn)
    # print(df)
    dfs.append(df)
    with pd.ExcelWriter(path='nba_stats.xlsx', engine='openpyxl', mode='a', if_sheet_exists='replace') as writer:
       df.to_excel(writer, sheet_name= sn, index=False, header=True)

In [112]:
# Combine all df and sorted by Win Rate
win_rate = 0.70 

full_df = pd.concat([x for x in dfs], ignore_index=True)
full_df = full_df.sort_values(by="WIN%", ascending= False).reset_index(drop=True)
full_df["Rank"] = full_df.index + 1

wol = ["W" if x >= win_rate else "L" for x in full_df["WIN%"]]
full_df.insert(7, "W/L", wol)

# Export df 
with pd.ExcelWriter(path='nba_stats.xlsx', engine='openpyxl', mode='a', if_sheet_exists='replace') as writer:
    full_df.to_excel(writer, sheet_name= 'Sheet', index=False, header=True)
    full_df.to_csv('nba_stats.csv', index=False)

In [91]:
# Close all opened files and systems
wb.close()
driver.close()

Statistics Below

In [121]:
print(f"Size of Dataframe {full_df.shape}")
full_df.describe()

Size of Dataframe (832, 30)


,Rank,GP,W,L,WIN%,Min,PTS,FGM,FGA,FG%,...,DREB,REB,AST,TOV,STL,BLK,BLKA,PF,PFD,+/-
count,832.000000,832.000000,832.000000,832.000000,832.000000,832.000000,832.000000,832.000000,832.000000,832.000000,...,832.000000,832.000000,832.000000,832.000000,832.000000,832.000000,832.000000,832.000000,832.000000,832.000000
mean,416.500000,79.235577,39.617788,39.617788,0.499820,48.347596,101.411178,37.763942,83.026563,45.466827,...,31.444952,42.636178,22.435817,14.516947,7.663822,4.912620,4.913341,20.985216,14.116587,-0.005048
std,240.322006,6.964050,12.641805,12.574613,0.152513,0.177342,7.741515,2.671618,4.261351,1.719039,...,2.344658,2.091804,2.372022,1.218248,0.887065,0.816421,0.706291,1.805163,9.426436,4.621914
min,1.000000,50.000000,7.000000,9.000000,0.106000,48.000000,81.900000,30.800000,71.200000,40.100000,...,24.900000,35.800000,15.600000,11.100000,5.500000,2.400000,3.000000,15.800000,0.000000,-13.900000
25%,208.750000,82.000000,30.000000,30.000000,0.390000,48.200000,95.675000,35.900000,79.800000,44.300000,...,29.700000,41.200000,20.800000,13.700000,7.100000,4.300000,4.400000,19.700000,0.200000,-3.100000
50%,416.500000,82.000000,41.000000,39.000000,0.512000,48.400000,99.900000,37.400000,82.600000,45.400000,...,31.250000,42.600000,22.100000,14.500000,7.600000,4.900000,4.900000,20.900000,19.500000,0.300000
75%,624.250000,82.000000,49.000000,49.000000,0.610000,48.500000,106.600000,39.500000,86.200000,46.600000,...,33.100000,44.000000,23.900000,15.300000,8.200000,5.400000,5.400000,22.200000,20.800000,3.300000
max,832.000000,82.000000,73.000000,72.000000,0.890000,49.000000,122.700000,46.700000,94.400000,50.500000,...,42.200000,51.700000,30.600000,19.000000,12.000000,8.200000,6.900000,27.100000,25.700000,11.600000


Data Cleaning. <br/>
Extract chosen columns from full df as stated in data and model report. <br/> 
Normalize the numeric cols that aids in winning the tournament. <br/>
